In [1]:
import pandas as pd
import numpy as np

In [2]:
#val_results = pd.read_csv('../doc/models/model_02/model_02.csv')
val_results = pd.read_csv('../../doc/models/model_02r/model_02r2.csv')
#val_results = pd.read_csv('../../doc/models/model_02s/model_02s.csv')
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score
0,sub-100079,ses-V02,1,T2w,sub-100079_ses-V02_run-1_T2w.nii.gz,3.0,1.344599
1,sub-100247,ses-V02,1,T2w,sub-100247_ses-V02_run-1_T2w.nii.gz,3.0,NaN
2,sub-100524,ses-V02,1,T2w,sub-100524_ses-V02_run-1_T2w.nii.gz,1.0,NaN
3,sub-100586,ses-V02,1,T2w,sub-100586_ses-V02_run-1_T2w.nii.gz,0.0,0.334282
4,sub-100642,ses-V02,1,T2w,sub-100642_ses-V02_run-1_T2w.nii.gz,0.0,NaN


In [3]:
# find scans with highest prediction error
val_results['abs_error'] = abs(val_results['QU_motion'] - val_results['predicted_qu_motion_score'])
val_results = val_results.sort_values(by='abs_error', ascending=False)
val_results.head(15)

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
171,sub-118437,ses-V03,2,T2w,sub-118437_ses-V03_run-2_T2w.nii.gz,3.0,0.603155,2.396845
263,sub-130428,ses-V04,1,T2w,sub-130428_ses-V04_run-1_T2w.nii.gz,3.0,0.825118,2.174882
265,sub-130428,ses-V04,1,T1w,sub-130428_ses-V04_run-1_T1w.nii.gz,3.0,0.986481,2.013519
264,sub-130428,ses-V04,2,T2w,sub-130428_ses-V04_run-2_T2w.nii.gz,3.0,1.164887,1.835113
787,sub-181584,ses-V04,1,T2w,sub-181584_ses-V04_run-1_T2w.nii.gz,3.0,1.244479,1.755521
0,sub-100079,ses-V02,1,T2w,sub-100079_ses-V02_run-1_T2w.nii.gz,3.0,1.344599,1.655401
653,sub-165452,ses-V02,1,T1w,sub-165452_ses-V02_run-1_T1w.nii.gz,1.0,2.424037,1.424037
468,sub-147780,ses-V02,1,T2w,sub-147780_ses-V02_run-1_T2w.nii.gz,3.0,1.603733,1.396267
369,sub-140160,ses-V02,2,T1w,sub-140160_ses-V02_run-2_T1w.nii.gz,3.0,1.725333,1.274667
706,sub-172232,ses-V02,1,T1w,sub-172232_ses-V02_run-1_T1w.nii.gz,0.5,1.678959,1.178959


In [6]:
# save first 15 rows to csv
val_results.head(15).to_csv('../../data/validation_errors/model_02r2/validation_set_top15_errors.csv', index=False)

In [7]:
# remove the 5 highest error scans
val_results = val_results.iloc[5:]
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
0,sub-100079,ses-V02,1,T2w,sub-100079_ses-V02_run-1_T2w.nii.gz,3.0,1.344599,1.655401
653,sub-165452,ses-V02,1,T1w,sub-165452_ses-V02_run-1_T1w.nii.gz,1.0,2.424037,1.424037
468,sub-147780,ses-V02,1,T2w,sub-147780_ses-V02_run-1_T2w.nii.gz,3.0,1.603733,1.396267
369,sub-140160,ses-V02,2,T1w,sub-140160_ses-V02_run-2_T1w.nii.gz,3.0,1.725333,1.274667
706,sub-172232,ses-V02,1,T1w,sub-172232_ses-V02_run-1_T1w.nii.gz,0.5,1.678959,1.178959


In [8]:
# count NaN values in abs_error column
val_results['abs_error'].isna().sum()

np.int64(817)

In [9]:
# drop rows with NaN values in abs_error column
val_results = val_results.dropna(subset=['abs_error'])
val_results['abs_error'].isna().sum()

np.int64(0)

In [10]:
def create_correlation_coefficient(actual_vals, predicted_vals):
    """Calculate Pearson correlation coefficient between actual and predicted values.

    Args:
        actual_vals (list): Actual QU_motion scores
        predicted_vals (list): Predicted QU_motion scores

    Returns:
        float: Pearson correlation coefficient (r value)
    """
    x = np.array(actual_vals)
    y = np.array(predicted_vals)

    correlation_matrix = np.corrcoef(x, y)
    correlation_coefficient = correlation_matrix[0, 1]

    return correlation_coefficient

In [11]:
# recalculate metrics without the 5 highest error scans
corr = create_correlation_coefficient(val_results['QU_motion'], val_results['predicted_qu_motion_score'])
corr

np.float64(0.834569159604421)